In [1]:
import socket
from _thread import *
from pynput import keyboard
import time
import os
import threading
import sys

bLoopEnd = False

def MainLoop():
    if(bLoopEnd == False):
        threading.Timer(0.1, MainLoop).start()
    else:
        print('Server close')
        server_socket.close()
        sys.exit()

def on_press(key):
    '''
    try:
        print('Alphanumeric key pressed: {0} '.format(key.char))
    except AttributeError:
        print('special key pressed: {0}'.format(key))
    '''     

def on_release(key):
    global bLoopEnd
    if key == keyboard.Key.esc:
        print("=====esc======")
        bLoopEnd = True;
        return False
    
def ClientChkThreaded(i):
    while True:
        print("ClientChkThreaded wait!!")
        client_socket, addr = server_socket.accept()
        
        print("client_socket connected!!", addr)
        threaded(client_socket, addr)

def threaded(client_socket, addr):
    print('Connected by : ', addr[0], ':', addr[1])

    while True:
        try:
            data = client_socket.recv(1024)
            if not data:
                print('Disconnected by' + addr[0], ':', addr[1])
                break
            print('Received from' + addr[0], ':', addr[1], data.decode())
            client_socket.send(data)
            
        except ConnectionResetError as e:
            print('Disconnected by' + addr[0], ':', addr[1])
            break
            
    print("end")
    client_socket.close() 

In [3]:
HOST = '127.0.0.1'
PORT = 9999
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind((HOST, PORT))
server_socket.listen()

print('server start')

i = 0
start_new_thread(ClientChkThreaded,(i,))
MainLoop()

with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
    listener.join()
# 또다른 리스너 등록 방법
#listener = keyboard.Listener(on_press=on_press, on_release=on_release)
#listener.start()
#listener.join()

while True:
    print('wait')
    client_socket, addr = server_socket.accept()
    start_new_thread(threaded, (client_socket, addr))

if on_press(key) == 'esc':
    server_socket.close()
    
server_socket.close()

server start
Server close


SystemExit: 

ClientChkThreaded wait!!


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
Exception ignored in thread started by: <function ClientChkThreaded at 0x00000211E376FA60>
Traceback (most recent call last):
  File "<ipython-input-1-adb676cba237>", line 42, in ClientChkThreaded
OSError: [WinError 10038] An operation was attempted on something that is not a socket


In [ ]:
import socket
from _thread import *
from pynput import keyboard
import time
import os
import threading
import sys

################################################################ 예외사항 처리 추가
bServerLoopEnd = False
bClientWaitEnd = False
bLoopEnd = False

def MainLoop():
    if(bServerLoopEnd == False):
        threading.Timer(0.1, MainLoop).start()
    else:
        print('Server close')
        server_socket.close()
        sys.exit()

def on_press(key):
    '''
    try:
        print('Alphanumeric key pressed: {0} '.format(key.char))
    except AttributeError:
        print('special key pressed: {0}'.format(key))
    '''     

def on_release(key):
    global bClientWaitEnd
    if key == keyboard.Key.esc:
        print("=====esc======")
        bClientWaitEnd = True;
        return False
    
def ClientChkThreaded(i):
    global bClientWaitEnd
    global bServerLoopEnd
    while True:
        print("ClientChkThreaded wait!!")
        server_socket.settimeout(1)
        if(bClientWaitEnd == False):
            try :
                client_socket, addr = server_socket.accept()
            except socket.timeout:
                continue
            server_socket.settimeout(None)
            print("client_socket connected!!", addr)
            threaded(client_socket, addr)
        else :
            print("KeyboardInterrupt")
            bServerLoopEnd = True
            break
################################################################

def threaded(client_socket, addr):
    print('Connected by : ', addr[0], ':', addr[1])

    while True:
        try:
            data = client_socket.recv(1024)
            if not data:
                print('Disconnected by' + addr[0], ':', addr[1])
                break
            print('Received from' + addr[0], ':', addr[1], data.decode())
            client_socket.send(data)
            
        except ConnectionResetError as e:
            print('Disconnected by' + addr[0], ':', addr[1])
            break
            
    print("end")
    client_socket.close() 
    
HOST = '127.0.0.1'
PORT = 9999
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind((HOST, PORT))
server_socket.listen()

print('server start')

i = 0
start_new_thread(ClientChkThreaded,(i,))
MainLoop()

with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
    listener.join()
# 또다른 리스너 등록 방법
#listener = keyboard.Listener(on_press=on_press, on_release=on_release)
#listener.start()
#listener.join()

while True:
    print('wait')
    client_socket, addr = server_socket.accept()
    start_new_thread(threaded, (client_socket, addr))

if on_press(key) == 'esc':
    server_socket.close()
    
server_socket.close()